### 1) 라이브러리 설치

In [6]:
%pip install -q transformers datasets sentencepiece

Note: you may need to restart the kernel to use updated packages.


#### Hugging Face Tokenizer
* Hugging Face 토크나이저를 LangChain의 Text Splitter와 함께 활용 가능

In [1]:
%pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
print(torch.__version__)  # 설치된 PyTorch 버전 확인

2.6.0+cpu


##### 1. Model Hub에서 GPT-2 모델 불러오기

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Hugging Face에서 GPT-2 모델과 토크나이저 다운로드
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# 입력 텍스트 토크나이징
input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors="pt")

# 모델 예측 실행
output = model.generate(**inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a


##### 2. 감성 분석 (Sentiment Analysis)

In [5]:
from transformers import pipeline
# 50256은 **GPT-2의 종료 토큰(End of Sequence, EOS)**을 의미합니다.
# 텍스트의 다양성 조절 temperature=0.7 → 적당한 창의성과 일관성 유지함
generator = pipeline("text-generation", model="gpt2", pad_token_id=50256, temperature=0.7)
# GPT-2가 "Hugging Face is"라는 문장을 보고, 이를 로맨스 소설과 연결하여 이야기를 만들었습니다.
print(generator("Hugging Face is", truncation=True))

Device set to use cpu


[{'generated_text': 'Hugging Face is available to download from Amazon.\n\nThis is the final edition of the first book to be released by the company.\n\nThe book will be available for pre-order at Amazon.\n\nThe book will be available for'}]


##### 3. IMDB 영화 리뷰 데이터셋 로드 & 미리보기

In [7]:
from datasets import load_dataset

dataset = load_dataset("imdb")
print(dataset["train"][0])  # 첫 번째 데이터 샘플 출력

# "Titanic" 영화에 대한 리뷰 찾기
titanic_reviews = [review for review in dataset["train"] if "Titanic" in review["text"]]

# 상위 3개 리뷰 출력
for review in titanic_reviews[:3]:
    print(review)

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

##### 4. 토크나이징 실행 (BERT 기반)

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokens = tokenizer.tokenize("Hugging Face is awesome!")
print(tokens)
# 출력 예: ['hugging', 'face', 'is', 'awesome', '!']

['hugging', 'face', 'is', 'awesome', '!']


##### 5. Gradio로 간단한 감성 분석 앱 만들기

* "I love this product! It's amazing!"  → (예상 출력: POSITIVE)
* "This movie was terrible. I hated it." → (예상 출력: NEGATIVE)

In [9]:
import gradio as gr
from transformers import pipeline

# pipeline("sentiment-analysis")는 기본적으로 IMDB 감성 분석 모델을 사용합니다.
#classifier = pipeline("sentiment-analysis")
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


def analyze_sentiment(text):
    return classifier(text)[0]["label"]

demo = gr.Interface(fn=analyze_sentiment, inputs="text", outputs="label")
demo.launch()

Device set to use cpu


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### Hugging Face Tokenizer
* Hugging Face 토크나이저를 LangChain의 Text Splitter와 함께 활용 가능

In [10]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter

# GPT-2 모델의 토크나이저 로드
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# 데이터 파일 읽기
file_path = "./data/ai-terminology.txt"
with open(file_path, encoding="utf-8") as f:
    file_content = f.read()

print("📄 원본 텍스트 미리보기:\n", file_content[:200])

# CharacterTextSplitter 설정 (Hugging Face 토크나이저 사용)
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    hf_tokenizer,
    chunk_size=300,  # 각 청크 크기 (토큰 기준 아님)
    chunk_overlap=50,  # 청크 간 중복 부분
)

# 텍스트 분할 수행
split_texts = text_splitter.split_text(file_content)

# 분할된 텍스트 출력
print(f"\n총 {len(split_texts)}개의 청크로 분할됨\n")
for i, chunk in enumerate(split_texts[:5]):  # 처음 5개만 출력
    print(f"Chunk {i+1} ({len(chunk)}자):\n{chunk}\n")

# 토크나이저로 텍스트를 토큰 단위로 변환하여 확인
tokenized_example = hf_tokenizer.tokenize(split_texts[0])
print(f"\n첫 번째 청크의 토큰 개수: {len(tokenized_example)}")
print("첫 번째 청크의 토큰 리스트:", tokenized_example[:20])  # 앞 20개만 출력

Created a chunk of size 321, which is longer than the specified 300
Created a chunk of size 362, which is longer than the specified 300


📄 원본 텍스트 미리보기:
 Semantic Search (의미론적 검색)

정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

FAISS (Faceboo

총 23개의 청크로 분할됨

Chunk 1 (25자):
Semantic Search (의미론적 검색)

Chunk 2 (157자):
정의: 사용자의 질의를 단순한 키워드 매칭이 아니라 문맥과 의미를 분석하여 관련 정보를 반환하는 검색 방식.
예시: "우주 탐사"를 검색하면 "아폴로 11호", "화성 탐사 로버"와 같은 연관 정보가 포함된 결과를 제공함.
연관 키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Chunk 3 (37자):
FAISS (Facebook AI Similarity Search)

Chunk 4 (176자):
정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Chunk 5 (143자):
Embedding (임베딩)

정의: 단어나 문장을 벡터 공간에 매핑하여 의미적으로 유사한 것들이 가까이 위치하도록 하는 기법.
예시: "강아지"와 "고양이"의 벡터 표현이 유사하게 위치함.
연관 키워드: 벡터화, 자연어 처리, 딥러닝

Token (토큰)


첫 번째 청크의 토큰 개수: 23
첫 번째 청크의 토큰 리스트: ['Sem', 'antic', 'ĠSearch', 'Ġ(', 'ìĿ', 'ĺ', 'ë', '¯', '¸', 'ë', '¡', 'ł', 'ì', 'ł', 'ģ', 'Ġ', 'ê', '²'

In [11]:
from transformers import GPT2TokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter, TokenTextSplitter

# ✅ Hugging Face의 GPT-2 토크나이저 로드
hf_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

# ✅ 분석할 영어 문장 (한글 파일 대신 직접 입력)
text = """Natural Language Processing (NLP) is a subfield of artificial intelligence (AI) that focuses on the interaction between computers and humans through natural language.
NLP enables machines to read, understand, and derive meaning from human languages through various techniques such as tokenization, parsing, and sentiment analysis."""

print("📄 원본 텍스트:\n", text, "\n")

# **문맥을 유지하는 RecursiveCharacterTextSplitter 사용**
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,  # 각 청크 크기 (토큰 기준이 아니라 의미 단위 기준)
    chunk_overlap=10,  # 중복된 텍스트 유지 (문맥 연결을 위해)
    separators=["\n\n", "\n", " ", ""],  # 문단 → 문장 → 단어 순서로 나눔
)
split_texts_char = text_splitter.split_text(text)

print(f"문자 단위 분할 결과 (총 {len(split_texts_char)}개 청크)\n")
for i, chunk in enumerate(split_texts_char[:5]):
    print(f"Chunk {i+1}: {chunk}")

# **토큰(Token) 단위 분할**
text_splitter_token = TokenTextSplitter(
    chunk_size=15,  # 각 청크 크기 (토큰 기준)
    chunk_overlap=3,  # 중복되는 토큰 수
)
split_texts_token = text_splitter_token.split_text(text)

print(f"\n토큰 단위 분할 결과 (총 {len(split_texts_token)}개 청크)\n")
for i, chunk in enumerate(split_texts_token[:5]):
    print(f"Chunk {i+1}: {chunk}")

# **첫 번째 청크의 토큰화 결과 확인**
tokenized_example = hf_tokenizer.tokenize(split_texts_token[0])
print(f"\n토큰 단위 첫 번째 청크의 토큰 개수: {len(tokenized_example)}")
print("첫 번째 청크의 토큰 리스트:", tokenized_example)

📄 원본 텍스트:
 Natural Language Processing (NLP) is a subfield of artificial intelligence (AI) that focuses on the interaction between computers and humans through natural language.
NLP enables machines to read, understand, and derive meaning from human languages through various techniques such as tokenization, parsing, and sentiment analysis. 

문자 단위 분할 결과 (총 8개 청크)

Chunk 1: Natural Language Processing (NLP) is a subfield of
Chunk 2: of artificial intelligence (AI) that focuses on
Chunk 3: on the interaction between computers and humans
Chunk 4: humans through natural language.
Chunk 5: NLP enables machines to read, understand, and

토큰 단위 분할 결과 (총 5개 청크)

Chunk 1: Natural Language Processing (NLP) is a subfield of artificial intelligence (
Chunk 2:  artificial intelligence (AI) that focuses on the interaction between computers and humans through
Chunk 3:  and humans through natural language.
NLP enables machines to read, understand
Chunk 4:  read, understand, and derive meaning from huma

##### 한국어 특화된 모델(KcBERT, KoBERT) 사용

In [12]:
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 한국어 지원 토크나이저 사용 (kcbert-base)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")  

# 예제 문장 (한글 포함)
text = "자연어 처리는 인공지능의 핵심 기술 중 하나입니다. NLP 모델은 의미를 이해하고 텍스트를 생성할 수 있습니다."

# 적절한 청크 크기 조정 (너무 작지 않게 설정)
splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer, 
    chunk_size=20,  # 더 큰 청크 크기
    chunk_overlap=5  # 문맥 유지를 위한 오버랩
)

# 텍스트 분할
split_texts = splitter.split_text(text)

# 실행 결과 출력
print(f"총 {len(split_texts)}개 청크로 분할됨\n")
for i, chunk in enumerate(split_texts):
    print(f"Chunk {i+1}: {chunk}\n")

# 첫 번째 청크의 토큰화 결과
tokens = tokenizer.tokenize(split_texts[0])
print(f"첫 번째 청크의 토큰 개수: {len(tokens)}")
print("첫 번째 청크의 토큰 리스트:", tokens)


총 7개 청크로 분할됨

Chunk 1: 자연어 처리는 인공지능의

Chunk 2: 인공지능의 핵심 기술 중

Chunk 3: 중 하나입니다. NLP

Chunk 4: NLP 모델은 의미를

Chunk 5: 의미를 이해하고 텍스트를

Chunk 6: 텍스트를 생성할 수

Chunk 7: 수 있습니다.

첫 번째 청크의 토큰 개수: 7
첫 번째 청크의 토큰 리스트: ['자연', '##어', '처리', '##는', '인공', '##지능', '##의']
